Tuba Siddiqui 2047057
 and Erica Capocello 1862113

#**DATA COLLECTION**
##***1.1. Get the list of places***


In [ ]:
import lxml 
import os 
import requests 
import urllib.request 
import time 
import codecs 
import pandas as pd 
import csv 
from bs4 import BeautifulSoup 
from tqdm import tqdm 
from datetime import datetime
from google.colab import drive 
import urllib.request, urllib.error, urllib.parse
from dateutil.parser import parse
import datetime
import pandas
import spacy

In [ ]:
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
link = open("/content/drive/MyDrive/ADM/places.txt", "w")

for i in tqdm(range(1,4)): 
  url = 'https://www.atlasobscura.com/places?page=' + str(i) + '&sort=likes_count'
  response = requests.get(url)
  soup = BeautifulSoup(response.content, features = 'lxml')
  for a in soup.find_all('a', class_ ="content-card content-card-place"):
    link.write(a.get('href')+ '\n')

link.close()


100%|██████████| 3/3 [00:02<00:00,  1.30it/s]


##***1.2. Crawl places***

In [ ]:
link_read = open("/content/drive/MyDrive/ADM/places.txt", "r")
folder = "page"
list_path=[]
for i, j  in enumerate(link_read): 
  num = i/18

  if type(num) == float:
    folder_num = int(num)+1

  else:
    folder_num = int(num)
  if i%18 == 0:
    directory = folder + str(folder_num)
    parent_dir = "/content/drive/MyDrive/ADM/"
    path = parent_dir + directory+ '/'
    os.mkdir(path)
  
  page_name = j.replace('/places/', '')
  page_name = page_name.replace('\n', '')
  url = 'https://www.atlasobscura.com' + str(j)
  response = urllib.request.urlopen(url)
  webContent = response.read().decode('UTF-8')
  list_path.append(path + str(page_name)+'.html')

  f = open(path + str(page_name)+'.html', 'w')
  f.write(webContent)
  f.close

link.close()
print(list_path)

['/content/drive/MyDrive/ADM/page1/city-hall-station.html', '/content/drive/MyDrive/ADM/page1/highgate-cemetery.html', '/content/drive/MyDrive/ADM/page1/leadenhall-market.html', '/content/drive/MyDrive/ADM/page1/wave-organ.html', '/content/drive/MyDrive/ADM/page1/catacombes-de-paris.html', '/content/drive/MyDrive/ADM/page1/evolution-store.html', '/content/drive/MyDrive/ADM/page1/magowans-infinite-mirror-maze.html', '/content/drive/MyDrive/ADM/page1/underground-tunnels-los-angeles.html', '/content/drive/MyDrive/ADM/page1/secret-tiled-staircase.html', '/content/drive/MyDrive/ADM/page1/dream-house.html', '/content/drive/MyDrive/ADM/page1/the-museum-of-death-los-angeles-california.html', '/content/drive/MyDrive/ADM/page1/gods-own-junkyard.html', '/content/drive/MyDrive/ADM/page1/last-bookstore.html', '/content/drive/MyDrive/ADM/page1/the-ruins-of-st-dunstan-in-the-east-london-england.html', '/content/drive/MyDrive/ADM/page1/echo-park-time-travel-mart.html', '/content/drive/MyDrive/ADM/page

##***1.3 Parse downloaded pages***

In [ ]:
def place_name(files):
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')
  placeName=str(soup.h1.get_text())
  return placeName


In [ ]:
def place_tags(files):
  Tags=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')    
  for tag in soup.find_all('div', class_='DDPage__header-place-location'):
      s=str(tag.contents)
      s1=s.partition('title=')[2]
      s2=s1.replace(">Manhattan, New York</a>]", "")
      se=s2.replace('"','')
      Tags.append(se)
  return Tags

In [ ]:
def people_visited(files):
  People=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')  
  for div in soup.find_all('div', class_ ='title-md item-action-count'):
      People.append(str(div.get_text()))
      Peopleviseted=People[0]
      Peoplewant=People[-1]
      numPeopleVisited=int(Peopleviseted)
      numPeopleWant=int(Peoplewant)
  return numPeopleVisited
  

In [ ]:
def people_wanted(files):
  People=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')
  for div in soup.find_all('div', class_ ='title-md item-action-count'):
    People.append(str(div.get_text()))
    Peoplewant=People[-1]
    numPeopleWant=int(Peoplewant)
  return numPeopleWant

In [ ]:
def long_description(files):
  long=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml') 
  for long in soup.find_all('div', class_ = 'DDP__body-copy'):
    desc = long.get_text()
  return desc



In [ ]:
def short_description(files):
  shortd=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')  
  for short in soup.find_all('h3', class_='DDPage__header-dek'):
      shortd.append(str(short.get_text()))
      short=shortd[0]
  return short

In [ ]:
def nearby_places(files):
 placeNearby=[]
 f = open(str(files), 'r')
 contents = f.read()
 soup = BeautifulSoup(contents, 'lxml')
 for nearby in soup.find_all('div', class_="DDPageSiderailRecirc__item-title"):
   places=str(nearby.get_text())
   placeNearby.append(places)
 return placeNearby 


In [ ]:
def address(files):
  a = []
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, "html.parser")    
  addr = soup.find('address', class_ ="DDPageSiderail__address")
  addr = addr.get_text()
  addr = addr.replace("\n", "")
  return addr

In [ ]:
def alt(files):
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')   
  for addr in soup.find_all('div', class_="DDPageSiderail__coordinates js-copy-coordinates"):
      s=str(addr)
      s1=s.partition('data-coordinates=')[2]
      s2=s1.partition('>')
      placealt=(s2[0].split()[0])
      placeAlt1=placealt.replace('"','')
      placeAlt=float(placeAlt1.replace(',',''))
  return placeAlt

In [ ]:
def long(files):
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')   
  for addr in soup.find_all('div', class_="DDPageSiderail__coordinates js-copy-coordinates"):
      s=str(addr)
      s1=s.partition('data-coordinates=')[2]
      s2=s1.partition('>')  
      placelong=(s2[0].split()[1])
      placeL=placelong.replace('"','')
      placeLong=float(placeL)
  return placeLong

In [ ]:
def editor(files):
  editors=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')    
  for e in soup.find_all('li', class_ ="DDPContributorsList__item"):
    for spn in e.find_all('span'):
      s=str(spn.text)
      editors.append(s)
  return editors

In [ ]:
def pub_date(files):
    f = open(str(files), 'r')
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml') 
    for long in soup.find_all('div', class_="DDPContributor__name"):
      s1=str(long.get_text())
      date = parse(s1)
    return date.date()


In [ ]:
def related_list(files):
  RelatedList=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')    
  for releted in soup.find_all('h3', class_='Card__heading --content-card-v2-title js-title-content'):
      s=str(releted.contents)
      s1=s.partition('<span>')[2]
      s2=s1.replace("</span>,", "")
      se=s2.replace("n']",'')
      s3=se.replace("'\\","")
      RelatedList.append(s3)
      placeRelatedLists=RelatedList[9:13]

  return placeRelatedLists

In [ ]:
def related_places(files):
  RelatedPlaces=[]
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')    
  for releted in soup.find_all('h3', class_='Card__heading --content-card-v2-title js-title-content'):
      s=str(releted.contents)
      s1=s.partition('<span>')[2]
      s2=s1.replace("</span>,", "")
      se=s2.replace("n']",'')
      s3=se.replace("'\\","")
      RelatedPlaces.append(s3)
      placeRelatedPlaces=RelatedPlaces[5:9]

  return placeRelatedPlaces

In [ ]:
def url(files):
  f = open(str(files), 'r')
  contents = f.read()
  soup = BeautifulSoup(contents, "html.parser") 
  urls = []
  for link in soup.find_all('link'):
    urls.append(str(link.get('href')))
  return urls[18]


Create our dataframe

In [ ]:
placeName=[]
placeTags=[]
numPeopleVisited=[]
numPeopleWant=[]
placeDesc=[]
placeShortDesc=[]
placeNearby=[]
placeAddress=[]
placeAlt=[]
placeLong=[]
placeEditors=[]
placePubDate=[]
placeRelatedLists=[]
placeRelatedPlaces=[]
placeURL=[]


for i in list_path:
  placeName.append(place_name(i))
  placeTags.append(place_tags(i))
  numPeopleVisited.append(people_visited(i))
  numPeopleWant.append(people_wanted(i))
  placeDesc.append(long_description(i))
  placeShortDesc.append(short_description(i))
  placeNearby.append(nearby_places(i))
  placeAddress.append(address(i))
  placeAlt.append(alt(i))
  placeLong.append(long(i))
  placeEditors.append(editor(i))
  placePubDate.append(pub_date(i))
  placeRelatedLists.append(related_list(i))
  placeRelatedPlaces.append(related_places(i))
  placeURL.append(url(i))

In [ ]:
dictionary={
    'placeName':placeName,
    'placeTags':placeTags,
    'numPeopleVisited': numPeopleVisited,
    'numPeopleWant': numPeopleWant,
    'placeDesc': placeDesc,
    'placeShortDesc': placeShortDesc,
    'placeNearby': placeNearby,
    'placeAddress': placeAddress,
    'placeAlt': placeAlt,
    'placeLong': placeLong,
    'placeEditors': placeEditors,
    'placePubDate': placePubDate,
    'placeRelatedLists': placeRelatedLists,
    'placeRelatedPlaces': placeRelatedPlaces,
    'placeURL': placeURL}

Dataframe=pd.DataFrame.from_dict(dictionary)
print(Dataframe.head())
dataframe=Dataframe.to_csv('Dataframe.tsv', sep="\t")


with open('/content/drive/MyDrive/ADM/dataframe.tsv', 'w', newline='') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(Dataframe)

    for i in range(Dataframe.shape[0]):
      tsv_output.writerow(Dataframe.iloc[i])



             placeName                                          placeTags  \
0    City Hall Station     [See all things to do in Manhattan, New York.]   
1    Highgate Cemetery  [See all things to do in London, England.>Lond...   
2    Leadenhall Market  [See all things to do in London, England.>Lond...   
3       The Wave Organ  [See all things to do in San Francisco, Califo...   
4  Catacombes de Paris  [See all things to do in Paris, France.>Paris,...   

   numPeopleVisited  numPeopleWant  \
0              1838           8647   
1              2637           8233   
2              3158           7643   
3              2435           7441   
4              4475           7084   

                                           placeDesc  \
0  \n\nThe first New York City subway was built a...   
1  \n\nOpened in 1839, Highgate is one of London’...   
2  \n\nThe ornate 19th-century painted roof and c...   
3  \n\nLocated on a jetty in the San Francisco Ba...   
4  \n\nIn 2004, Parisian pol

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADM/dataframe.tsv", sep='\t')
df.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"['See all things to do in Manhattan, New York.']",1838,8647,\n\nThe first New York City subway was built a...,A beautiful and abandoned New York subway stat...,"['African Burial Ground National Monument', 'T...","31 Centre StNew York, New York, 10007United St...",40.7134,-74.0046,"['charding407', 'fosterc827', 'Mike H', 'erjef...",2010-05-08,['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway ', 'Moscow Metro Stati...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"['See all things to do in London, England.>Lon...",2637,8233,"\n\nOpened in 1839, Highgate is one of London’...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","Swain's Lane, HighgateLondon, England, N6Unite...",51.5675,-0.1483,"['thesecretcity', 'thedyerghoulhouse', 'juliap...",2014-08-09,"['Znojmo Catacombs ', ""The World's Top 100 Won...","['Isokon Flats ', 'Jewett City Vampires ', 'To...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"['See all things to do in London, England.>Lon...",3158,7643,\n\nThe ornate 19th-century painted roof and c...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England, EC3VUnited Kingdom",51.5126,-0.0834,"['Julius Spada', 'Katecrimson', 'thesecretcity...",2016-08-01,"['Drvengrad ', 'The Ultimate Guide to Stunning...","['Signs of Old Lombard Street ', 'Rivendell ',...",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"['See all things to do in San Francisco, Calif...",2435,7441,\n\nLocated on a jetty in the San Francisco Ba...,A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","83 Marina Green DrSan Francisco, California, 9...",37.8085,-122.4401,"['routt1111', 'wefiddleboy14', 'Saal333', 'Nei...",2008-11-21,"['Gunnar Schonbeck Exhibit ', ""Leonardo Nam's ...","['Aquatic Park Tombstones ', 'Sea Organ ', 'Si...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"['See all things to do in Paris, France.>Paris...",4475,7084,"\n\nIn 2004, Parisian police were assigned to ...","The vast, legendary catacombs hold secrets muc...","['Sculptures de Décure', 'Arago Medallions', ""...","1 Place Denfert-RochereauParis, 75014France",48.8343,2.3322,"['marypippen', 'ramonrodz2212', 'mathewbrowne'...",2009-02-13,"['Melník Chapel of Bones ', '19 Catacombs Sure...","['Montparnasse Cemetery ', 'Ossario di San Mar...",https://www.atlasobscura.com/places/catacombes...


Create a tsv file for each places

In [ ]:
files = []
import os
# assign directory
folder_count = 0  # type: int
input_path = "/content/drive/MyDrive/ADM/"  # type: str
for folders in os.listdir(input_path):  # loop over all files
    if os.path.isdir(os.path.join(input_path, folders)):  # if it's a directory
        folder_count += 1  # increment counter
print(folder_count)
for i in range(1,folder_count):
  directory = "/content/drive/MyDrive/ADM/page" + str(i)
 
  # iterate over files in
  # that directory
  for filename in os.listdir(directory):
      f = os.path.join(directory, filename)
      # checking if it is a file
      if os.path.isfile(f):
          files.append(f)

4


In [ ]:
import csv

lst = []
for name in files:

  lst.append(place_name(name))
  lst.append(place_tags(name))
  lst.append(people_visited(name))
  lst.append(people_wanted(name))
  lst.append(long_description(name))
  lst.append(short_description(name))
  lst.append(nearby_places(name))
  lst.append(alt(name))
  lst.append(long(name))
  lst.append(pub_date(name))
  lst.append(editor(name))
  lst.append(address(name))
  lst.append(related_list(name))
  lst.append(related_places(name))
  lst.append(url(name))
  path = '/content/drive/MyDrive/ADM/TSV/'+str(place_name(name))+'.tsv'
  with open(path, 'w', newline='') as f_output:
      tsv_output = csv.writer(f_output, delimiter='\t')
      tsv_output.writerow(lst)

##***2. Search Engine***


* Removing the stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def stop_words(s):
  s1=s.lower()
  s_words=[]
  stop_words = set(stopwords.words('english')) 
  words = s1.split()
  for r in words: 
    if not r in stop_words: 
        s_words.append(r)
  return s_words

In [ ]:
stop_words('The first day of the month')

['first', 'day', 'month']

* Removing punctuation

In [ ]:
def punctuation(s):
  string_no_punct = s.translate(str.maketrans('', '', string.punctuation))
  return string_no_punct


* Stemming

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
nlp=spacy.load('en_core_web_sm')
def stem_text(txt):
  doc=nlp(txt)
  return [token.lemma_ for token in doc if not token.is_punct and token.pos_ not in ['DET', 'CCONJ']]

Create a new dataframe where we remove the stopwords, the punctuation and stem the text.

In [ ]:
def create_dataframe(df):
  column=['placeName','placeTags','placeDesc','placeShortDesc','placeNearby','placeAddress','placeEditors', 'placeRelatedLists','placeRelatedPlaces','placeURL']
  dfNew = {key: None for key in column}
  for i in column:
    dfNew[i] = df.apply(lambda row: punctuation(row[i]), axis=1)
    dfNew[i] = df.apply(lambda row: stem_text(row[i]), axis=1)
    dfNew[i] = df.apply(lambda row: stop_words(row[i]), axis=1)
  return dfNew



In [ ]:
dfNew=create_dataframe(df)

In [ ]:
dfx = pd.DataFrame.from_dict(dfNew)
dfx.placeDesc
dfx['ID']=range(dfx.shape[0])


##***2.1.1 Conjunctive query***

In [ ]:
from collections import Counter
from functools import reduce

We want to build the vocabulary that we are going to use in order to assign a unique term_id to each word.

In [ ]:
uniqueWord = [];
for i in range(dfx.shape[0]):
  for j in dfx.placeDesc[i]:
    if j not in uniqueWord:
        uniqueWord.append(j)
vocabulary={}
index_id=[]
for i in range(len(uniqueWord)):
  vocabulary[uniqueWord[i]]=i

print(vocabulary)



{'first': 0, 'new': 1, 'york': 2, 'city': 3, 'subway': 4, 'built': 5, 'operated': 6, 'interborough': 7, 'rapid': 8, 'transit': 9, 'company': 10, '(irt)': 11, 'opened': 12, 'october': 13, '27,': 14, '1904,': 15, 'joy': 16, 'elevated': 17, 'train': 18, 'streetcar': 19, 'riders.': 20, 'hall': 21, 'station': 22, 'irt': 23, 'local': 24, 'track': 25, 'lavished': 26, 'fine': 27, 'architectural': 28, 'details,': 29, 'including': 30, 'glass': 31, 'tiles': 32, 'large': 33, 'chandeliers.': 34, 'however,': 35, 'gustavino': 36, 'vaulted': 37, 'ceilings': 38, 'skylights': 39, 'lost': 40, 'busy': 41, 'commuters,': 42, 'stop': 43, 'one': 44, 'least-used': 45, 'system.': 46, 'turnstiles': 47, 'installed': 48, '1923,': 49, 'nearby': 50, 'brooklyn': 51, 'bridge': 52, 'frequented': 53, 'express': 54, 'closer': 55, 'connecting': 56, 'streetcars.': 57, 'curved': 58, 'platform,': 59, 'cars': 60, 'center': 61, 'doors': 62, 'could': 63, 'used': 64, 'unless': 65, 'specially': 66, 'modified': 67, 'door': 68, 'co

In [ ]:
voc_list=list(vocabulary.values())

2.1.1) Create your index!

We want to create an inverted index, in terms of dictionary:

{ term_id_1:[document_1, document_2, document_4], term_id_2:[document_1, document_3, document_5, document_6], ...}

It will allow us to know, for each term_id, which are the documents that contain the corresponding word.

In [ ]:
def inverted_index(vocab):
  inverted_index = {}
  for word, id in vocab.items():
    inverted_index[id] = list(dfx[dfx.placeDesc.apply(lambda row: word in row)].index)
  return inverted_index
inverted_index(vocabulary)


{0: [0, 1, 4, 19, 23, 29, 30, 33, 37, 38, 46, 50, 53],
 1: [0, 9, 10, 12, 17, 18, 19, 20, 26, 28, 32, 36, 37, 44, 47, 50, 51],
 2: [0, 18, 19, 36, 37, 47, 50],
 3: [0, 1, 4, 7, 13, 18, 22, 26, 28, 30, 31, 32, 39, 41, 47],
 4: [0, 7],
 5: [0, 3, 4, 13, 15, 17, 18, 19, 26, 30, 31, 36, 47, 49],
 6: [0, 53],
 7: [0],
 8: [0],
 9: [0],
 10: [0, 14, 17, 23, 28, 29, 33],
 11: [0],
 12: [0, 1, 4, 18, 19, 23, 26, 29, 34, 50],
 13: [0, 4, 31],
 14: [0],
 15: [0],
 16: [0],
 17: [0, 22, 44],
 18: [0, 7, 51],
 19: [0],
 20: [0],
 21: [0, 7],
 22: [0, 30],
 23: [0],
 24: [0, 1, 7, 11, 14, 26, 30, 31, 32, 39, 48, 49],
 25: [0],
 26: [0],
 27: [0],
 28: [0, 47],
 29: [0],
 30: [0, 8, 13, 19, 26, 32, 36, 38, 50],
 31: [0, 5, 27, 36, 37, 40, 53],
 32: [0],
 33: [0, 7, 17, 23, 30, 37, 45, 47, 53],
 34: [0],
 35: [0, 1, 2, 18, 19, 20, 24, 28, 32, 39, 47],
 36: [0],
 37: [0, 15],
 38: [0],
 39: [0, 19],
 40: [0, 14, 17],
 41: [0, 4, 22, 29, 47],
 42: [0, 51],
 43: [0, 27],
 44: [0,
  1,
  2,
  3,
  4,
  6

2.1.2) Execute the query


In [ ]:
query='new york'

In [ ]:
def Query(query):
  query_out={}
  l=[]
  q=[]
  p=[]
  list_query=list(query.split())
  for i in list_query:
    for j in range(dfx.shape[0]):
      if i in dfx.placeDesc[j]:
        l.append(dfx.ID[j])

  for c in range(len(list_query)):
    c=list(Counter(l).values())
    for d in range(len(c)):
      if c[d] ==len(list_query):
        q.append(l[d])

  for number in q:
     query_out[number]=df[['placeName','placeDesc', 'placeURL']].iloc[number]
  queryout=pd.DataFrame.from_dict(query_out)
  return queryout.T
Query(query)

,placeName,placeDesc,placeURL
0,City Hall Station,\n\nThe first New York City subway was built a...,https://www.atlasobscura.com/places/city-hall-...
18,Roosevelt Island Smallpox Hospital Ruins,\n\nFew diseases have had a greater impact on ...,https://www.atlasobscura.com/places/roosevelt-...
19,Eastern State Penitentiary,\n\nThrust into the middle of urban Philadephi...,https://www.atlasobscura.com/places/eastern-st...
36,Explorers Club Headquarters,\n\nBehind impressive heavy doors and ornate t...,https://www.atlasobscura.com/places/explorers-...
37,Mapparium,"\n\nIn the early 1930s, Boston architect Chest...",https://www.atlasobscura.com/places/mapparium
47,5 Beekman Street,"\n\nIn New York City, where space seems to be ...",https://www.atlasobscura.com/places/5-beekman-...
50,Lexington Candy Shop,\n\nFor those looking for a taste of old New Y...,https://www.atlasobscura.com/places/the-lexing...


##***2.2) Conjunctive query & Ranking score***

In [ ]:
import numpy as np
import math
from numpy.linalg import norm 
import sys

We can define the <b>IDF</b>, inverse document frequency, in this way:</p>

<p>$IDF = log_{10}\left(\frac{N_d}{n_d}\right)$</p>

<p>where $N_d$ is the total number of documents and $n_d$ the number of documents containing the word</p>

In [ ]:
def idx(query):
  all=Query(query)
  num_doc=all.shape[0]
  idf=1+np.log10(num_doc/dfx.shape[0])
  return idf

idx(query)

0.11270428019128831

The TF_Q is the relative frequency of the words in the query

In [ ]:
def tf_query(query):
  true_query=list(query.split())
  freq=[]
  for i in range(len(true_query)):
      freq_ass=list(Counter(true_query).values())
  for i in freq_ass:
    freq.append(i/len(true_query))
  
  return freq


tf_query(query)

[0.5, 0.5]

The TF_D is the relative frequency of the words of the query in each document

In [ ]:
def tf_doc(query):
  query_out={}
  l=[]
  q=[]
  p=[]
  list_query=list(query.split())
  for i in list_query:
    for j in range(dfx.shape[0]):
      if i in dfx.placeDesc[j]:
        l.append(dfx.ID[j])

  for c in range(len(list_query)):
    c=list(Counter(l).values())
    for d in range(len(c)):
      if c[d] ==len(list_query):
        q.append(l[d])

  for number in q:
     query_out[number]=dfx[['placeDesc']].iloc[number]
  queryout=pd.DataFrame.from_dict(query_out)
  dataf=queryout.T

  freq=[]
  freq_rel=[]
  c=[]
  for j in range(len(list_query)):
    for i in range(dataf.shape[0]):
      freq.append(Counter(dataf.placeDesc.iloc[i])[list_query[j]])
  for j in freq:
    freq_rel.append(j/len(list_query))
  freq_re=[freq_rel[i:i + len(list_query)] for i in range(0, len(freq_rel), len(list_query))]
  
  return freq_re


print(tf_doc(query))

[[1.5, 0.5], [0.5, 0.5], [1.0, 1.0], [1.5, 1.5], [0.5, 0.5], [0.5, 0.5], [1.0, 1.0]]


The tfidf correspond to:

$TFIDF = TF_D \cdot IDF$

In [ ]:
def tfidf(query): 
  tf_d=np.array(tf_doc(query))
  idx_out=idx(query)
  tfidf_d=np.dot(tf_d, idx_out)
  tf_p=np.array(tf_query(query))
  tfidf_p=np.dot(tf_p, idx_out)
  return tfidf_q,tfidf_d
query = 'new york'
tfidf_q,tfidf_d = tfidf(query)
print(tfidf_d)

[[0.16905642 0.05635214]
 [0.05635214 0.05635214]
 [0.11270428 0.11270428]
 [0.16905642 0.16905642]
 [0.05635214 0.05635214]
 [0.05635214 0.05635214]
 [0.11270428 0.11270428]]


We compute the similarity between the query and each one of the document as the cosine:</p>

$ cos(\theta) = (\vec{q} \cdot \vec{d})/(|{\vec{q}}| \cdot |{\vec{d}}|)$


In [ ]:
def cos_similarity(query):
    tfidf_q,tfidf_d=tfidf(query)
    cos=[]
    num=[]
    den=[]
    all=Query(query)


    for i in range(len(tfidf_d)):
      num.append(np.dot(tfidf_d[i], np.array(tfidf_q)))
      den.append(np.linalg.norm(tfidf_d[i])*np.linalg.norm(tfidf_q))
      cos.append(num[i]/den[i])

    return cos

print(cos_similarity(query))


[0.8944271909999157, 0.9999999999999999, 0.9999999999999999, 1.0, 0.9999999999999999, 0.9999999999999999, 0.9999999999999999]


2.2.1) Inverted index

In [ ]:
tfidf_q,tfidf_d = tfidf(query)
inverted_index2={}

query='new york'
list_id=[]
list_tuple=[]
id=[]
true_query=list(query.split())
for i in true_query:
  if i in vocabulary.keys():
    list_id.append(vocabulary[str(i)])



for i in range(len(tfidf_d.T)):
  for j in range(len(tfidf_d.T[i])):
    list_tuple.append((all.index[j],tfidf_d.T[i][j]))

output=[list_tuple[i:i +  all.shape[0]] for i in range(0, len(list_tuple), all.shape[0])]


for i in range(len(list_id)):
  inverted_index2[i]=[]
  inverted_index2[i]=(output[i])
print(inverted_index2)




{0: [(0, 0.16905642028693246), (18, 0.056352140095644154), (19, 0.11270428019128831), (36, 0.16905642028693246), (37, 0.056352140095644154), (47, 0.056352140095644154), (50, 0.11270428019128831)], 1: [(0, 0.056352140095644154), (18, 0.056352140095644154), (19, 0.11270428019128831), (36, 0.16905642028693246), (37, 0.056352140095644154), (47, 0.056352140095644154), (50, 0.11270428019128831)]}


2.2.2) Execute the query

In [ ]:
k=10
sim=cos_similarity(query)
all['similarity']=sim
all_sort=all.sort_values(by=['similarity'], ascending=False)
all_sort.head(k)



#***6. Command line question***



*   How many places can be found in each country?

grep -c "United States" dataframe.tsv

grep -c "Spain" dataframe.tsv

grep -c "France" dataframe.tsv

grep -c "England" dataframe.tsv

* How many people, on average, have visited the places in each country?

cut -f4,5,8 dataframe.tsv | grep "Italy" > Italy.tsv | awk '{s+=$1} END {print s/NR}' Italy.tsv

cut -f4,5,8 dataframe.tsv | grep "Spain" > Spain.tsv |awk '{s+=$1} END {print s/NR}' Spain.tsv

cut -f4,5,8 dataframe.tsv | grep "France" > France.tsv | awk '{s+=$1} END {print s/NR}' France.tsv

cut -f4,5,8 dataframe.tsv | grep "England" > England.tsv | awk '{s+=$1} END {print s/NR}' Englend.tsv

cut -f4,5,8 dataframe.tsv | grep "United States" > US.tsv | awk '{s+=$1} END {print s/NR}' US.tsv


* How many people in total want to visit the places in each country?

cut -f4,5,8 dataframe.tsv | grep "Italy" > Italy.tsv | awk '{s+=$2} END {print s}' Italy.tsv

cut -f4,5,8 dataframe.tsv | grep "Spain" > Spain.tsv |awk '{s+=$2} END {print s}' Spain.tsv

cut -f4,5,8 dataframe.tsv | grep "France" > France.tsv | awk '{s+=$2} END {print s}' France.tsv

cut -f4,5,8 dataframe.tsv | grep "England" > England.tsv | awk '{s+=$2} END {print s}' Englend.tsv

cut -f4,5,8 dataframe.tsv | grep "United States" > US.tsv | awk '{s+=$2} END {print s}' US.tsv


**7 Theoratical Question**

Sorting

Insertion Sort

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [ ]:
data = open('ApplicantsInfo.txt').readlines()

In [ ]:
m = 1000
n = 50000

def averg(stds):
    marks = [int(i) for i in stds if i.isdigit()]
    avg = sum(marks) / len(marks)
    return avg

avg=[]
for i in range(1, n):
    stds=data[i].split()
    line=stds[0:2]
    line.append(averg(stds))
    avg.append(line)
    

In [ ]:
N = len(avg)

In [ ]:

def InsertionSort(avg, N):
    for i in (range(N)):
            c=0
            while (c<i and avg[c][1] >= avg[i][1]): 
                if (avg[c][1] == avg[i][1]) & (avg[c][0] > avg[i][0]):
                    avg[c], avg[i] = avg[i], avg[c]
                c+=1
            temp=avg[i]  
            avg.insert(c,temp)
            c+=1
    return avg
start=time.time()
InsertionSort(avg, N)
end=time.time()
totalIS= start - end

$\textbf{Time complexity}$

$T(N)=\sum_{i=1}^N i$

$T(N)= \frac{(N+1)k}{2}$

$T(N)=O(N^2)$

Bubble Sort

In [ ]:
def BubbleSort(avg, N):
    for i in range(N): 
        for j in range(i): 
            if avg[j][2] >= avg[j+1][2]: 
                if avg[j][2] == avg[j+1][2]: 
                    if avg[j][0] >= avg[j+1][0]: 
                        if avg[j][0] == avg[j+1][0]:
                            if avg[j][1] > avg[j+1][1]:
                                avg[j] , avg[j+1] = avg[j+1] , avg[j] 
                        else:
                            avg[j] , avg[j+1] = avg[j+1] , avg[j] 
                else:
                    avg[j] , avg[j+1] = avg[j+1] , avg[j]
    return avg

start=time.time()
BubbleSort(avg, N)
end=time.time()
totalBS= start - end

$\textbf{Time complexity}$

$T(N)=\sum_{i=1}^N N-i$

$T(N)=\sum_{i=1}^N i$

$T(N)= \frac{(N+1)N}{2} $

$T(N)=O(N^2)$



Selection Sort

In [ ]:
def SelectionSort(avg, N):
    for i in range(N):
        lw = i 
        for j in range(i+1, N):
            if avg[j][1]> avg[lw][1]:
                lw =j
            if avg[j][1]== avg[lw][1]:
                    if avg[j][0] < avg[lw][0]:
                        temp1 = avg[j]
                        avg[j] = avg[lw]
                        avg[lw] = temp1
        avg[i], avg[lw] = avg[lw], avg[i]
    return avg

start=time.time()
SelectionSort(avg, N)
end=time.time()
totalSS= start - end

$\textbf{Time complexity}$

$T(N)=\sum_{i=1}^N N-i $

$T(N)=\sum_{i=1}^N i$

$T(N)= \frac{(N+1)N}{2}$

$T(N)=O(N^2)$

$\textbf{Evaluate the time taken for each of your implementations to answer the query stored in the ApplicantsInfo.txt file and visualize them.}$

In [ ]:
data = {'Insertion Sort':totalIS, 'Bubble Sort':totalBS, 'Selection Sort':totalSS}
fig = plt.figure(figsize = (10, 5))
plt.bar(data.keys(), data.values(), width = 0.4)

$\textbf{What is the most optimal algorithm, in your opinion, and why?}$

Comparing Selection sort and insertion sort, insertion sort is better since we can change positions many time. 
Bubble sort gets even slower if the dataset is huge sor it is worse then both.
